In [ ]:
# !pip install ffmpeg-python

Collecting ffmpeg-python

  Downloading ffmpeg_python-0.2.0-py3-none-any.whl (25 kB)
  
Requirement already satisfied: future in c:\programdata\anaconda3\lib\site-packages (from ffmpeg-python) (0.18.2)

Installing collected packages: ffmpeg-python

Successfully installed ffmpeg-python-0.2.0

In [ ]:
import ffmpeg
import os

In [ ]:
source = "TestFile.mp3"
output = "OutFile.mp3"

In [ ]:
spd = 1.5 # output speed
vol = 1.0 # volume rise ratio if any
bit = 16 # BitRate: 16K
frq = 24000 # sampling stream frequency

#Temporary parameters
src, out = source, output
# commands
stg1 = f'ffmpeg -i {src} -filter:a "atempo={spd},volume={vol}" -q:a 100 {out}'
stg2 = f"ffmpeg -i {src} -ac 1 -b:a {bit}k -ar {frq} -write_xing 0 {output}"